# Nova Feature para um Site

* Um portal de notícias implementou um play automático de vídeos em suas páginas, visando manter seus usuários por mais tempo no site.
* Analise os dados históricos contendo os acessos do último mês, e mostre se houve um aumento significativo de tempo de navegação após a implementação da feature.
* Arquivos: 
  * Populacao_tempo.csv
  * Amostra_tempo.csv
*  **Sabendo que o custo por visualização desse play é de 0.005 centavos, e que cada minuto adicional de navegação gera em média 5 centavos de lucro, defina se essa nova feature deve ser mantida ou não.**


In [39]:
# setup
import scipy as sp
from scipy import stats

import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls

py.sign_in('phboueke', '')

In [31]:
# loading data
population = []
sample = []

with open('data/populacao_tempo.csv') as file:
    file.readline()
    for line in file:
        time = float(line.split(';')[1][:-1])
        population.append(time)

        
population = sp.array(population)
popMean = sp.mean(population)
popSd = sp.std(population)

with open('data/amostra_tempo.csv') as file:
    file.readline()
    for line in file:
        time = float(line.split(';')[1][:-1])
        sample.append(time)

sample = sp.array(sample)
sampleMean = sp.mean(sample)

Iremos fazer um teste de hipótese para descobrir se a feature deve ser mantida avaliando os tempos de permanência antes da alteração (population) e depois (sample). A hipótese que queremos decidir primeiramente é a de que a feature impactou nos tempos de uso do site.

In [32]:
result = stats.ttest_ind(population, sample)
print(result)
mean_diff = sampleMean - popMean
print("Mean diff: " + str(mean_diff))

Ttest_indResult(statistic=-23.325754825312675, pvalue=2.6163856353001462e-120)
Mean diff: 1.47376129705


O valor-p obtido nos permite rejeitar H0. Vamos agora calcular a viabilidade financeira da feature sabendo que ela impactou positivamente o tempo de permanencia médio no site.

In [33]:
income_min = 0.05
cost_use = 0.005

gain = (mean_diff * income_min) - cost_use

print(gain)

0.0686880648525


Ok. A feature deve ser mantida, visto que calculamos um valor de lucro esperado de quase 7 centavos! Além desse fato, podemos fazer um box plot para ter uma boa ideia da distribuição dos tempos antes e depois das mudanças.

In [36]:
popsample = np.random.choice(population, len(sample))

before = go.Box(
    y=popsample
)
after = go.Box(
    y=sample
)
data = [before, after]
#py.iplot(data)
tls.embed("https://plot.ly/~phboueke/86/") #use this to embed the graph at the static page

# Clicks do Site

* Uma métrica comum em sites de e-commerce é o número de clicks que um usuário efetua durante a navegação. Um grupo de marketing quer fazer uma campanha de um novo produto, entretanto não sabe se apresenta ele apenas na página ou como um pop-up, que toma a conta de toda a tela. Visando responder esse problema, dois grupos foram selecionados. Para o primeiro grupo, foi apresentado apenas a tela com o produto. Para o segundo, foi apresentado a tela com o pop-up.
* Utilize técnicas estatísticas para informar se faz diferença utilizar o pop-up ou não.
* Arquivos:
  * amostra_A_click.csv
  * amostra_B_click.csv


In [42]:
# loading data
A = []
B = []

with open('data/amostra_A_click.csv') as file:
    file.readline()
    for line in file:
        if "y" in line:
            A.append(1)
        else:
            A.append(0)

with open('data/amostra_B_click.csv') as file:
    file.readline()
    for line in file:
        if "y" in line:
            B.append(1)
        else:
            B.append(0)
            
A = sp.array(A)
A_mean = sp.mean(A)
B = sp.array(B)
B_mean = sp.mean(B)

In [47]:
# let's plot!

trace0 = go.Bar(
    x = ["yes", "no"],
    y = [sum(A), len(A)-sum(A)],
    name = "A",
)
trace1 = go.Bar(
    x = ["yes", "no"],
    y = [sum(B), len(B)-sum(B)],
    name = "B",
)

data = [trace0, trace1]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig)
tls.embed("https://plot.ly/~phboueke/92/")

Sendo o dataset "B" aquele coletado utilizando popup, parece claro que o uso do popup aumentou o número de clicks no produto. Não obstante, faremos o teste da hipótese de que a utilização do popup resultou em diferenças entre os datasets.

In [48]:
result = stats.ttest_ind(A, B)
print(result)
print("A mean: " + str(A_mean) + ", B mean: " + str(B_mean))

Ttest_indResult(statistic=-3.6564760626262478, pvalue=0.00026234731820501704)
A mean: 0.306205493388, B mean: 0.383928571429


O valor-p obtido nos permite afirmar que a utilização do popup resultou em uma diferença estatisticamente significante. A vizualização, por sua vez, nos permite afirmar que essa diferença foi positiva em favor da utilização do popup. Podemos ver, também, por meio das médias calculadas, que os percentuais de clicks e "não clicks" em cada dataset.